<a href="https://colab.research.google.com/github/ameyj17/XAI/blob/main/Omni_XAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install omnixai

In [ ]:
pip install omnixai[plot]

In [ ]:
pip install omnixai[vision]

In [ ]:
import json
import torch
from torchvision import models, transforms
from PIL import Image as PilImage

from omnixai.data.image import Image
from omnixai.explainers.vision.specific.gradcam.pytorch.gradcam import GradCAM
from omnixai.explainers.vision import IntegratedGradientImage
from omnixai.explainers.vision import LimeImage
import torch.nn as nn



img = Image(PilImage.open('/content/frame_20190905112522_x_0003494.jpg').convert('RGB'))

with open('/content/gdrive/MyDrive/MultimodalOD/ORIGNAL/auair.json', 'r') as read_file:
    class_idx = json.load(read_file)
    idx2label = [class_idx[str(k)][1] for k in range(len(class_idx))]

model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/gdrive/MyDrive/DSTMMDI/XAIModels/Weights/best.pt')
model.requires_grad_(True)


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
requirements: YOLOv5 requirement "setuptools>=65.5.1" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2023-3-2 Python-3.8.10 torch-1.13.1+cu116 CPU

Fusing layers... 
Model summary: 157 layers, 7031701 parameters, 0 gradients
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [ ]:
for param in model.parameters():
    param.requires_grad = True

In [ ]:
# The preprocessing model
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
preprocess = lambda ims: torch.stack([transform(im.to_pil()) for im in ims])



target_layer = models.yolo.Detect
explainer = IntegratedGradientImage(
    model=model,
    target_layer=target_layer,
    preprocess_function=preprocess
)

# Explain the top label
explanations = explainer.explain(img)
explanations.ipython_plot(index=0, class_names=idx2label)

# Given the generated explanations, we launch a dashboard (a Dash app) for visualization.
dashboard = Dashboard(
    instances=img,
    local_explanations=local_explanations,
    class_names=idx2label,
)
dashboard.show()

In [ ]:
# The preprocessing model
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
preprocess = lambda ims: torch.stack([transform(im.to_pil()) for im in ims])



target_layer = model.model.model.model[-2]
explainer = IntegratedGradientImage(
    model=model,
    target_layer=target_layer,
    preprocess_function=preprocess
)

# Explain the top label
explanations = explainer.explain(img)
explanations.ipython_plot(index=0, class_names=idx2label)

# Given the generated explanations, we launch a dashboard (a Dash app) for visualization.
dashboard = Dashboard(
    instances=img,
    local_explanations=local_explanations,
    class_names=idx2label,
)
dashboard.show()

# **LIME**

In [ ]:
# Load the test image
img = Image(PilImage.open('/content/frame_20190905112522_x_0003494.jpg').convert('RGB'))
# Load the class names
with open('/content/gdrive/MyDrive/MultimodalOD/ORIGNAL/auair.json', 'r') as read_file:
    class_idx = json.load(read_file)
    idx2label = [class_idx[str(k)][1] for k in range(len(class_idx))]

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/gdrive/MyDrive/DSTMMDI/XAIModels/Weights/best.pt')
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
requirements: YOLOv5 requirement "setuptools>=65.5.1" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2023-3-2 Python-3.8.10 torch-1.13.1+cu116 CPU

Fusing layers... 
Model summary: 157 layers, 7031701 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
model.eval()
input_img = transform(img.to_pil()).unsqueeze(dim=0)
probs_top_5 = torch.nn.functional.softmax(model(input_img), dim=1).topk(5)
r = tuple((p, c, idx2label[c]) for p, c in
          zip(probs_top_5[0][0].detach().numpy(), probs_top_5[1][0].detach().numpy()))
print(r)

# Feature MAP

In [ ]:
import os
import torch
from torchvision import models, transforms
from PIL import Image as PilImage

from omnixai.data.image import Image
from omnixai.explainers.vision.specific.feature_visualization.visualizer import \
    FeatureMapVisualizer

In [ ]:
img = Image(PilImage.open("/content/frame_20190905112522_x_0003494.jpg").convert("RGB"))

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/gdrive/MyDrive/DSTMMDI/XAIModels/Weights/best.pt')
transform = transforms.Compose(
    [

        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)
preprocess = lambda ims: torch.stack([transform(im.to_pil()) for im in ims])

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
requirements: YOLOv5 requirement "setuptools>=65.5.1" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2023-3-2 Python-3.8.10 torch-1.13.1+cu116 CPU

Fusing layers... 
Model summary: 157 layers, 7031701 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
explainer = FeatureMapVisualizer(
    model=model,
    target_layer=model.model.model.model[-2],
    preprocess_function=preprocess
)
explanations = explainer.explain(img)
explanations.ipython_plot()

# Integrated Gradients

In [ ]:
import json
import torch
from torchvision import models, transforms
from PIL import Image as PilImage

from omnixai.data.image import Image
from omnixai.explainers.vision import IntegratedGradientImage

In [ ]:
# Load the test image
img = Image(PilImage.open('/content/frame_20190905112522_x_0003494.jpg').convert('RGB'))
# Load the class names
with open('/content/gdrive/MyDrive/MultimodalOD/ORIGNAL/auair.json', 'r') as read_file:
    class_idx = json.load(read_file)
    idx2label = [class_idx[str(k)][1] for k in range(len(class_idx))]

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/gdrive/MyDrive/DSTMMDI/XAIModels/Weights/best.pt')
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
preprocess = lambda ims: torch.stack([transform(im.to_pil()) for im in ims])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2023-3-2 Python-3.8.10 torch-1.13.1+cu116 CPU

Fusing layers... 
Model summary: 157 layers, 7031701 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='path/to/best.pt')

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
preprocess = lambda ims: torch.stack([transform(im.to_pil()) for im in ims])

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
requirements: YOLOv5 requirement "setuptools>=65.5.1" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2023-3-2 Python-3.8.10 torch-1.13.1+cu116 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
explainer = IntegratedGradientImage(
    model=model,
    preprocess_function=preprocess
)
# Explain the top label
explanations = explainer.explain(img)
explanations.ipython_plot(index=0, class_names=idx2label)